# How to migrate from legacy LangChain agents to LangGraph



Memory is just persistence, aka checkpointing.

Add a ```checkpointer``` to the agent and you get chat memory for free.

Persistance: https://langchain-ai.github.io/langgraph/how-tos/persistence/

CheckPoints: https://langchain-ai.github.io/langgraph/reference/checkpoints/

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# Basic Usage

For basic creation and usage of a tool-calling ReAct-style agent, the functionality is the same. First, let's define a model and tool(s), then we'll use those to create an agent.

In [2]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2


tools = [magic_function]


query = "what is the value of magic_function(3)?"

# Agent Scratch Pad

For the LangChain AgentExecutor, we define a prompt with a placeholder for the agent's scratchpad. The agent can be invoked as follows:

In [4]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        # First put the history
        ("placeholder", "{chat_history}"),
        # Then the new input
        ("human", "{input}"),
        # Finally the scratchpad
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [7]:
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [8]:
agent_executor.invoke({"input": query})

{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of `magic_function(3)` is 5.'}

In [9]:
# Function to generate a unique session ID
import uuid

def generate_session_id() -> str:
    return str(uuid.uuid4())

In [10]:
# Generating a dynamic session ID
session_id_1 = generate_session_id()

This way is not recommended for real worls. Refer to previous notebook for persistence history in a store{} rather than in memory Chat history

In [11]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

memory = InMemoryChatMessageHistory(session_id=session_id_1)


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [12]:
config = {"configurable": {"session_id": session_id_1}}


In [13]:
print(
    agent_with_chat_history.invoke(
        {"input": "Hi, I'm polly! What's the output of magic_function of 3?"}, config
    )["output"]
)

The output of the magic_function with an input of 3 is 5.


In [14]:
print(agent_with_chat_history.invoke({"input": "Remember my name?"}, config)["output"])


Yes, your name is Polly! How can I assist you today?


In [15]:
print(
    agent_with_chat_history.invoke({"input": "what was that output again?"}, config)[
        "output"
    ]
)

The output of the magic_function with an input of 3 is 5.


# LangGraph's react agent executor

LangGraph's react agent executor manages a state that is defined by a list of messages. It will continue to process the list until there are no tool calls in the agent's output. To kick it off, we input a list of messages. The output will contain the entire state of the graph-- in this case, the conversation history.
https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent



In [ ]:
from langgraph.prebuilt import create_react_agent

app = create_react_agent(model, tools)


messages = app.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}